In [1]:
from bs4 import BeautifulSoup
from parse_utils import *
from url_utils import get_urls_from_afisha_and_online_cinema
import requests
import logging
import multiprocessing.dummy as mp
import os


Получаем все ссылки на фильмы c афишы и онлайн кинотеатра

In [2]:


logging.basicConfig(level=logging.INFO)    
base_url_afisha = 'https://afisha.tut.by/film/'
base_url_online_cinema='https://afisha.tut.by/online-cinema/'
urlsOfFilms=get_urls_from_afisha_and_online_cinema(base_url_afisha,base_url_online_cinema)
logging.info("Number of films "+str(len(urlsOfFilms)))

INFO:root:Number of films 3277



Пишем функцию для парсинга страницы

In [3]:
def parse_film_page(url):
    logging.info(url)
    response_of_film_page = requests.get(url)
    parsed_film_page = BeautifulSoup(response_of_film_page.text, 'html.parser')
    name = get_film_name(parsed_film_page)
    data = {'url': url, 'name': name}  # set name and url of film
    data['is_premiere'] = is_premiere(parsed_film_page)  # is this film premiere
    data['age_rank'] = get_film_age_rank(parsed_film_page)  # age rank  (16+ and etc.)
    data['native_film_name'] = get_native_film_name(parsed_film_page)  # name of film in original
    data['poster_url'] = get_poster_url(parsed_film_page)  # url to film poster
    data['genres'] = get_genre(parsed_film_page)  # genres of film
    data['year'] = get_year_of_film(parsed_film_page)  # year of creation
    data['duration'] = get_duration(parsed_film_page)  # duration
    data['countries'] = get_country(parsed_film_page)  # countries 
    data['showing_end_date'] = get_showing_end_date(parsed_film_page)  # end date of showing in theaters
    data['desription'] = get_description(parsed_film_page)  # text description of film
    data['ratings'] = get_ratings(parsed_film_page)  # ratings
    data['directors'] = get_director(parsed_film_page)  # directors of film and link for additional director information
    data['actors'] = get_actors(parsed_film_page)  # actors  and link for additional actors information
    data['film_videos'] = get_film_videos(parsed_film_page)  # urls to videos of film (trailer and etc.)
    data['film_images'] = get_film_images(parsed_film_page)  # urls to images of film 
    return name, data


Пишем функцию для парсинга страниц

In [4]:
def parse_all_pages(urls):
    cores_num=os.cpu_count()
    logging.info("***************************START*****************************")
    p=mp.Pool(cores_num)
    parsed_pages=dict(p.map(parse_film_page,urls))
    logging.info("***************************END*****************************")
    return parsed_pages


Парсим страницы

In [5]:
data=parse_all_pages(urlsOfFilms)
logging.info("Prepared data for "+str(len(data))+" films")


INFO:root:***************************START*****************************
INFO:root:https://afisha.tut.by/film/derzhat_distanciyu_67432/
INFO:root:https://afisha.tut.by/film/voydite_strazhdushchie_48116/
INFO:root:https://afisha.tut.by/film/muzhchina_kotorogo_slishkom_silno_lyubili/
INFO:root:https://afisha.tut.by/film/novogodniy_brak_69527/
INFO:root:https://afisha.tut.by/film/korol_belgiycev/
INFO:root:https://afisha.tut.by/film/garazh/
INFO:root:https://afisha.tut.by/film/muzhchina_dlya_molodoy_zhenshchiny_67499/
INFO:root:https://afisha.tut.by/film/mezhdu-nami-devochkami/
INFO:root:https://afisha.tut.by/film/privet-poka-88699/
INFO:root:https://afisha.tut.by/film/maksimka_47909/
INFO:root:https://afisha.tut.by/film/sirop/
INFO:root:https://afisha.tut.by/film/tri_plyus_dva/
INFO:root:https://afisha.tut.by/film/chernaya_voda/
INFO:root:https://afisha.tut.by/film/chernaya_vual_69052/
INFO:root:https://afisha.tut.by/film/rodina_69706/
INFO:root:https://afisha.tut.by/film/pokhishchenie-ev

Сохраняем словарь в файл

In [6]:
my_file = open("../result/result.txt", "w", encoding='utf8')
for key,val in data.items():
    my_file.write('{}:{}\n'.format(key,str(val)))
my_file.close()




